In [1]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import dotenv
import os
from dotenv import load_dotenv


In [ ]:
#set up twitter api access

# get the API access keys from file .env

load_dotenv()  # take environment variables from .env.
config = dotenv.dotenv_values(".env")   # config = {"USER": "foo", "EMAIL": "foo@example.org"}

print(config)

# keys and tokens from the Twitter Dev Console
consumer_key = config.get('consumer_key')
consumer_secret = config.get('consumer_secret')
access_token = config.get('access_token')
access_token_secret = config.get('access_token_secret')

# create a twitter client class

In [3]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
#            fetched_tweets = self.api.search(q = query, count = count)
            fetched_tweets = self.api.search_tweets(q = query, count = count)

            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.TweepyException as e:
            # print error (if any)
            print("Error : " + str(e))


# create our main and search for query
# and save data to a file

In [4]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'TSLA', count = 2000)
  
    # printing first 12  tweets
    print("\n\nFirst 12 tweets:")
    for tweet in tweets[:12]:
        print(tweet['text'])
  
    # what is current directory?
    cwd = os.getcwd()
    print("Current directory = " + cwd)
    # save it to a file in data_sets directory
    file2 = open(r"./datasets/twitter_search_query_results.txt","w+")
    # write all tweets to the file
    for tweet in tweets:
        file2.write("{}\n".format(tweet))
    # close the file handle
    file2.close()
    
if __name__ == "__main__":
    # calling main function
    main()



First 12 tweets:
RT @garyblack00: For those who can’t get through Barron’s paywall.  $tsla $twtr https://t.co/VFQKaQaBzN
@KeepIt1K4L $TSLA has been a historically poor earnings overnight play. Move generally comes the following days.… https://t.co/oDPZHY8au2
@fh_404 I love you dev! @MercedeYanev @mat98nft @NitinRa42184046 @samostad @MatejFaganeli @hotdonna_eth @sethjpete… https://t.co/0Td7G6eG1a
$TSLA Reportedly Plans For 'Closed-Loop Production System' As Giga Shanghai Set To Restart. $XPEV https://t.co/WaQlu0tlwg
RT @EveryElonReply: Elon Musk liked a tweet from Tesla Owners Silicon Valley #Tesla $TSLA https://t.co/zA6Oh7FMex
#NativeDeFi on @defichain
👑DeFi anchored to King #Bitcoin 
🤑30-500+% APY for your #BTC  #ETH $LTC #dogecoin d-… https://t.co/Jb7znZCd0z
RT @charliebilello: Returns, Last 10 Years..
Bitcoin $BTC: +820,509%
Tesla $TSLA: +14,628%
NVIDIA $NVDA: +6,207%
Netflix $NFLX: +2,190%
Ama…
Tesla may finally be preparing to restart operations at its Giga Shanghai, with multip